In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors
import numpy as np
import os
import json
from ipywidgets import interact, widgets
import time
#plt.rcParams['text.usetex'] = True

import CuttingPlanes as CP

# Mixed Integer Linear Program (MILP)
$$
\begin{align}
&\min_{x} && c^T x\\
&\text{s.t.} && Ax \leq b\\
& && Bx = d\\
& && x \geq 0\\
& && x \in \mathbb{Z}^{n_1} \times \mathbb{R}^{n-n_1}
\end{align}
$$

In [33]:
# Specify Files
root_dir = "/Users/tobiaskohler/Uni/CuttingPlanes"
plots_dir = os.path.join(root_dir, 'plots')
data_dir = os.path.join(root_dir, 'data')
cpp_file = os.path.join(root_dir, 'build-CuttingPlanes-Desktop_arm_darwin_generic_mach_o_64bit-Debug/CuttingPlanes')
milp_files = ['milp_papadimitriou_14-2.json', 'milp_wolsey_8-3.json', 'infeasible.json']
milp_files = [os.path.join(data_dir, mf) for mf in milp_files]
milp_file = milp_files[2]
print(milp_file)
res_file = os.path.join(data_dir, 'cp.json')

# Create a MILP
milp = CP.MILP(2, 1) # dimension, number of integer constraints
#milp.set_objective([0, -1])
#milp.add_inequality_constraint([3, 2], 6)
#milp.add_inequality_constraint([-3, 1], 0)
#milp.save('milp.json')

# Load a MILP
milp.load(milp_file)

# Run Cutting Plane Algorithm
b = os.popen(' '.join([cpp_file, milp_file, res_file])).read()
print(b)

# Add Nonnegativity Constraints for Visualization
milp.invert_constraints() # Visualization expects ">=" constraints
milp.add_inequality_constraint([1,0], 0, True)
milp.add_inequality_constraint([0,1], 0, True)
milp.check_dimensions()

# Load the Cutting Planes Solution(s) of the Simplex Algorithm
data = json.load(open(res_file))
sols = data['sols']
A_cuts = -np.array(data['cuts_coeffs'], dtype=np.float64)
b_cuts = -np.array(data['cuts_rhs'], dtype=np.float64)
n_cuts = len(b_cuts)

/Users/tobiaskohler/Uni/CuttingPlanes/data/infeasible.json
Cutting Planes
================ Mixed Integer Linear Program ================
n, m, p, n1: 2, 1, 0, 1
c:
 0 -1
A:
1 1
b:
-1
B:

d:


Cut me some slack!



PROBLEM IS INFEASIBLE


In [31]:
# Save Plots
plotter = CP.Plotter()
imgs = []
for i in range(n_cuts+1):
    for j in range(2):
        title = milp.objective_str()
        img_file = os.path.join(plots_dir, '{i}{j}.png'.format(i=i,j=j))

        sol = None if j==0 and i==0 else sols[i-1] if j==0 else sols[i] 

        cuts = (np.array([[]]),np.array([])) if i==0 else (A_cuts[:i], b_cuts[:i])
        
        plotter.plot_milp(milp, title=title, optimal_solution=sol, cuts=cuts, save_file=img_file)

        img = mpimg.imread(img_file)
        imgs.append(img)

In [32]:
def show_plot(iter):
    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(imgs[iter])
    plt.show()

interact(show_plot, iter=widgets.IntSlider(min=0, max=2*n_cuts+1, step=1, value=0, description='Show Step'))

interactive(children=(IntSlider(value=0, description='Show Step', max=3), Output()), _dom_classes=('widget-int…

<function __main__.show_plot(iter)>